In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from config import user_name,password
import MySQLdb
from sqlalchemy import create_engine

In [2]:
#find all 30 Dow Jone Industrial stock tickers
url = "https://finance.yahoo.com/quote/%5EDJI/components?p=%5EDJI"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
results = soup.find_all('a', class_="C($c-fuji-blue-1-b) Cur(p) Td(n) Fw(500)")
tickers1=[]

for x in results:
    tickers1.append(x.text)



In [3]:
results1=soup.find_all('td', class_="Py(10px) Ta(start) Pend(10px)")
company_name=[]
for x in range(1,61,2):
    company_name.append(results1[x].text)
company_name

['The Procter & Gamble Company',
 'Verizon Communications Inc.',
 "McDonald's Corporation",
 'The Travelers Companies, Inc.',
 'The Coca-Cola Company',
 'Pfizer Inc.',
 'UnitedHealth Group Incorporated',
 '3M Company',
 'Exxon Mobil Corporation',
 'Chevron Corporation',
 'Merck & Co., Inc.',
 'Johnson & Johnson',
 'NIKE, Inc.',
 'The Walt Disney Company',
 'Walmart Inc.',
 'American Express Company',
 'The Home Depot, Inc.',
 'Visa Inc.',
 'Walgreens Boots Alliance, Inc.',
 'JPMorgan Chase & Co.',
 'Microsoft Corporation',
 'International Business Machines Corporation',
 'Intel Corporation',
 'The Goldman Sachs Group, Inc.',
 'Cisco Systems, Inc.',
 'Dow Inc.',
 'United Technologies Corporation',
 'Caterpillar Inc.',
 'The Boeing Company',
 'Apple Inc.']

In [4]:
Company_Name_Table=pd.DataFrame()
Company_Name_Table["Ticker"]=tickers1
Company_Name_Table["Company_Name"]=company_name
Company_Name_Table=Company_Name_Table.sort_values(by="Ticker")
Company_Name_Table=Company_Name_Table.reset_index(drop=True)
Company_Name_Table.insert(0,"Ticker_ID",Company_Name_Table.index+1)
Company_Name_Table.tail()

Ticker_ID Ticker                    Company_Name
25         26      V                       Visa Inc.
26         27     VZ     Verizon Communications Inc.
27         28    WBA  Walgreens Boots Alliance, Inc.
28         29    WMT                    Walmart Inc.
29         30    XOM         Exxon Mobil Corporation

In [5]:
tickers=tickers1.copy()
tickers.sort()


In [6]:
# URL of page to be scraped
url = "https://finance.yahoo.com/quote/WMT/key-statistics?p=WMT"
df=pd.DataFrame()
df1=pd.DataFrame()
df2=pd.DataFrame()
df3=pd.DataFrame()
df4=pd.DataFrame()
df5=pd.DataFrame()
df6=pd.DataFrame()
df7=pd.DataFrame()
df8=pd.DataFrame()
tables = pd.read_html(url)
df=pd.DataFrame(tables[0][0])

In [7]:
for x in tickers:
    url= f"https://finance.yahoo.com/quote/{x}/key-statistics?p={x}"
    k=pd.read_html(url)
    df[x]=k[0][1]
    df1[x]=k[7][1]
    df2[x]=k[8][1]
    df3[x]=k[9][1]
    df4[x]=k[2][1]
    df5[x]=k[4][1]
    df6[x]=k[5][1]
    df7[x]=k[6][1]
    df8[x]=k[3][1]

In [8]:
dfcopy=df.T.copy()
dfcopy.columns = dfcopy.iloc[0]
dfcopy=dfcopy[1:]
dfcopy.reset_index(level=0,inplace=True)
dfcopy.insert(0, "Ticker_ID", dfcopy.index+1)
Valuation_Measures_Table=dfcopy.rename(columns={"index":"Ticker"})
Valuation_Measures_Table.insert(1,"Company_Name", Company_Name_Table["Company_Name"].copy())
Valuation_Measures_Table=Valuation_Measures_Table.drop(["Ticker"], axis=1)
Valuation_Measures_Table.head()

0  Ticker_ID              Company_Name Market Cap (intraday) 5  \
0          1                Apple Inc.                 851.89B   
1          2  American Express Company                  96.64B   
2          3        The Boeing Company                 189.79B   
3          4          Caterpillar Inc.                  71.52B   
4          5       Cisco Systems, Inc.                  225.3B   

0 Enterprise Value 3 Trailing P/E Forward P/E 1 PEG Ratio (5 yr expected) 1  \
0            939.78B        15.58         14.44                        1.34   
1            127.01B        14.74         12.86                        1.59   
2            207.73B        19.31         14.95                        1.22   
3            105.31B        11.63          9.60                        0.56   
4            220.17B        18.67         15.14                        1.74   

0 Price/Sales (ttm) Price/Book (mrq) Enterprise Value/Revenue 3  \
0              3.30             8.06                       3.64   
1              2.57             4.36                       3.38   
2              1.89          1519.50                       2.06   
3              1.29             4.63                       1.90   
4              4.43             5.55                       4.33   

0 Enterprise Value/EBITDA 6  
0                     12.15  
1                       NaN  
2                     15.44  
3                      9.46  
4                     14.44

In [9]:
Stock_Price_Hist_Table = df1.T.copy() 
Stock_Price_Hist_Table.columns=tables[7][0]
Stock_Price_Hist_Table.reset_index(level=0,inplace=True)
Stock_Price_Hist_Table=Stock_Price_Hist_Table.rename(columns = lambda x : str(x)[:-1])
Stock_Price_Hist_Table=Stock_Price_Hist_Table.rename(columns={"Beta (3Y Monthly":"Beta (3Y Monthly)"})
Stock_Price_Hist_Table.insert(0, "Ticker_ID", dfcopy.index+1)
Stock_Price_Hist_Table=Stock_Price_Hist_Table.rename(columns={"inde":"Ticker"})
Stock_Price_Hist_Table.insert(1,"Company_Name", Company_Name_Table["Company_Name"].copy())
Stock_Price_Hist_Table=Stock_Price_Hist_Table.drop(["Ticker"], axis=1)
Stock_Price_Hist_Table.columns
Stock_Price_Hist_Table=Stock_Price_Hist_Table.rename(columns={'52-Week Change ':'52 Week Change',\
    'S&P500 52-Week Change ':'S&P500 52 Week Change','50-Day Moving Average ':'50 Day Moving Average',\
        '200-Day Moving Average ':'200 Day Moving Average','52 Week High ':'52 Week High',\
        '52 Week Low ':'52 Week Low'})
Stock_Price_Hist_Table.head()


0  Ticker_ID              Company_Name Beta (3Y Monthly) 52 Week Change  \
0          1                Apple Inc.              0.89          4.80%   
1          2  American Express Company              1.10         16.95%   
2          3        The Boeing Company              1.33          2.93%   
3          4          Caterpillar Inc.              1.33        -15.42%   
4          5       Cisco Systems, Inc.              1.05         16.76%   

0 S&P500 52 Week Change 52 Week High 52 Week Low 50 Day Moving Average  \
0                 5.54%       233.47      142.00                199.91   
1                 5.54%       119.66       89.05                113.29   
2                 5.54%       446.01      292.47                375.59   
3                 5.54%       161.60      112.06                137.58   
4                 5.54%        57.53       40.25                 55.14   

0 200 Day Moving Average  
0                 180.93  
1                 106.82  
2                 366.48  
3                 131.58  
4                  49.02

In [10]:
Share_Statistics_Table=df2.T.copy()
Share_Statistics_Table.columns=tables[8][0]
Share_Statistics_Table.reset_index(level=0,inplace=True)
Share_Statistics_Table=Share_Statistics_Table.rename(columns = lambda x : str(x)[:-1])
Share_Statistics_Table=Share_Statistics_Table.rename(columns=lambda x: x.strip())
Share_Statistics_Table=Share_Statistics_Table.rename(columns={"Floa":"Float"})
Share_Statistics_Table.insert(0, "Ticker_ID", dfcopy.index+1)
Share_Statistics_Table=Share_Statistics_Table.rename(columns={"inde":"Ticker"})
Share_Statistics_Table.insert(1,"Company_Name", Company_Name_Table["Company_Name"].copy())
Share_Statistics_Table=Share_Statistics_Table.drop(["Ticker"], axis=1)
Share_Statistics_Table.head()


0  Ticker_ID              Company_Name Avg Vol (3 month) Avg Vol (10 day)  \
0          1                Apple Inc.            27.76M           32.42M   
1          2  American Express Company             3.33M            3.63M   
2          3        The Boeing Company              7.8M            5.69M   
3          4          Caterpillar Inc.             4.13M            4.96M   
4          5       Cisco Systems, Inc.            20.29M           22.73M   

0 Shares Outstanding    Float % Held by Insiders % Held by Institutions  \
0               4.6B    4.34B              0.07%                 60.95%   
1            835.08M  681.35M              0.29%                 86.33%   
2            562.63M  562.23M              0.07%                 70.56%   
3            571.88M  570.87M              0.17%                 68.58%   
4               4.4B     4.4B              0.07%                 77.33%   

0 Shares Short (Apr 30, 2019) Short Ratio (Apr 30, 2019)  \
0                      52.67M                       2.19   
1                       6.38M                       1.94   
2                       5.31M                       0.79   
3                       6.67M                       1.67   
4                      39.89M                        2.4   

0 Short % of Float (Apr 30, 2019)  \
0                           1.11%   
1                           0.93%   
2                           0.94%   
3                           1.17%   
4                           0.86%   

0 Short % of Shares Outstanding (Apr 30, 2019)  \
0                                        1.14%   
1                                        0.76%   
2                                        0.94%   
3                                        1.17%   
4                                        0.91%   

0 Shares Short (prior month Mar 29, 2019)  
0                                  67.79M  
1                                    6.7M  
2                                   5.07M  
3                                   6.26M  
4                                  42.22M

In [11]:
Dividends_Splits_Table=df3.T.copy()
Dividends_Splits_Table.columns=tables[9][0]
Dividends_Splits_Table.reset_index(level=0,inplace=True)
Dividends_Splits_Table=Dividends_Splits_Table.rename(columns = lambda x : str(x)[:-2])
Dividends_Splits_Table.insert(0, "Ticker_ID", dfcopy.index+1)
Dividends_Splits_Table=Dividends_Splits_Table.rename(columns={"ind":"Ticker"})
Dividends_Splits_Table.insert(1,"Company_Name", Company_Name_Table["Company_Name"].copy())
Dividends_Splits_Table=Dividends_Splits_Table.drop(["Ticker"], axis=1)
Dividends_Splits_Table.head()


0  Ticker_ID              Company_Name Forward Annual Dividend Rate  \
0          1                Apple Inc.                         2.92   
1          2  American Express Company                         1.56   
2          3        The Boeing Company                         8.22   
3          4          Caterpillar Inc.                         3.44   
4          5       Cisco Systems, Inc.                          1.4   

0 Forward Annual Dividend Yield Trailing Annual Dividend Rate  \
0                         1.46%                          2.92   
1                         1.33%                          1.52   
2                         2.18%                          7.18   
3                         2.47%                          3.44   
4                         2.50%                          1.32   

0 Trailing Annual Dividend Yield 5 Year Average Dividend Yield Payout Ratio  \
0                          1.48%                          1.68       23.11%   
1                          1.28%                          1.45       19.36%   
2                          2.03%                          2.30       41.17%   
3                          2.62%                          3.00       31.20%   
4                          2.47%                          2.95       48.71%   

0 Dividend Date Ex-Dividend Date Last Split Factor (new per old)  \
0  May 16, 2019     May 10, 2019                             1/7   
1   Aug 9, 2019      Jul 3, 2019                      8753/10000   
2   Jun 7, 2019      May 9, 2019                             1/2   
3  Aug 20, 2019     Jul 19, 2019                             1/2   
4  Apr 24, 2019      Apr 4, 2019                             1/2   

0 Last Split Date  
0     Jun 9, 2014  
1     Oct 3, 2005  
2     Jun 9, 1997  
3    Jul 14, 2005  
4    Mar 23, 2000

In [12]:
Profitability_Table=df4.T.copy()
Profitability_Table.columns=tables[2][0]
Profitability_Table.reset_index(level=0,inplace=True)
Profitability_Table=Profitability_Table.rename(columns={"index":"Ticker"})
Profitability_Table.insert(0, "Ticker_ID", dfcopy.index+1)
Profitability_Table.insert(1,"Company_Name", Company_Name_Table["Company_Name"].copy())
Profitability_Table=Profitability_Table.drop(["Ticker"], axis=1)
Profitability_Table.head()


0  Ticker_ID              Company_Name Profit Margin Operating Margin (ttm)
0          1                Apple Inc.        22.12%                 25.34%
1          2  American Express Company        18.18%                 21.32%
2          3        The Boeing Company        10.06%                 11.24%
3          4          Caterpillar Inc.        11.50%                 15.20%
4          5       Cisco Systems, Inc.        25.31%                 26.46%

In [13]:
Management_Effectiveness_Table=df8.T.copy()
Management_Effectiveness_Table.columns=tables[2][0]
Management_Effectiveness_Table.reset_index(level=0,inplace=True)
Management_Effectiveness_Table=Management_Effectiveness_Table.rename(columns={"index":"Ticker"})
Management_Effectiveness_Table.insert(0, "Ticker_ID", dfcopy.index+1)
Management_Effectiveness_Table.insert(1,"Company_Name", Company_Name_Table["Company_Name"].copy())
Management_Effectiveness_Table=Management_Effectiveness_Table.drop(["Ticker"], axis=1)
Management_Effectiveness_Table.head()

0  Ticker_ID              Company_Name Profit Margin Operating Margin (ttm)
0          1                Apple Inc.        11.54%                 49.13%
1          2  American Express Company         3.63%                 32.69%
2          3        The Boeing Company         6.05%              1,324.44%
3          4          Caterpillar Inc.         6.71%                 41.40%
4          5       Cisco Systems, Inc.         7.19%                 27.74%

In [14]:
tables[4]

0        1
0                    Revenue (ttm)  514.41B
1          Revenue Per Share (ttm)   175.62
2   Quarterly Revenue Growth (yoy)    1.90%
3               Gross Profit (ttm)   129.1B
4                           EBITDA    32.8B
5   Net Income Avi to Common (ttm)    6.67B
6                Diluted EPS (ttm)     2.26
7  Quarterly Earnings Growth (yoy)   69.50%

In [15]:
Income_Statement_Table=df5.T.copy()
Income_Statement_Table.columns=tables[4][0]
Income_Statement_Table.reset_index(level=0,inplace=True)
Income_Statement_Table=Income_Statement_Table.rename(columns={"index":"Ticker"})
Income_Statement_Table.insert(0, "Ticker_ID", dfcopy.index+1)
Income_Statement_Table.insert(1,"Company_Name", Company_Name_Table["Company_Name"].copy())
Income_Statement_Table=Income_Statement_Table.drop(["Ticker"], axis=1)
Income_Statement_Table.head()

0  Ticker_ID              Company_Name Revenue (ttm) Revenue Per Share (ttm)  \
0          1                Apple Inc.       258.49B                   54.15   
1          2  American Express Company         37.6B                   44.15   
2          3        The Boeing Company       100.66B                  175.33   
3          4          Caterpillar Inc.        55.33B                   94.58   
4          5       Cisco Systems, Inc.        50.82B                   10.99   

0 Quarterly Revenue Growth (yoy) Gross Profit (ttm)  EBITDA  \
0                         -5.10%            101.84B  77.34B   
1                          6.80%             25.51B     NaN   
2                         -2.00%             19.64B  13.45B   
3                          4.70%              14.9B  11.13B   
4                          4.70%             30.68B  15.25B   

0 Net Income Avi to Common (ttm) Diluted EPS (ttm)  \
0                         57.17B             11.89   
1                           6.7B              7.85   
2                         10.13B             17.47   
3                          6.36B             10.75   
4                         12.87B              2.74   

0 Quarterly Earnings Growth (yoy)  
0                         -16.40%  
1                          -5.10%  
2                         -13.20%  
3                          13.00%  
4                             NaN

In [16]:
Balance_Sheet_Table=df6.T.copy()
Balance_Sheet_Table.columns=tables[5][0]
Balance_Sheet_Table.reset_index(level=0,inplace=True)
Balance_Sheet_Table=Balance_Sheet_Table.rename(columns={"index":"Ticker"})
Balance_Sheet_Table.insert(0, "Ticker_ID", dfcopy.index+1)
Balance_Sheet_Table.insert(1,"Company_Name", Company_Name_Table["Company_Name"].copy())
Balance_Sheet_Table=Balance_Sheet_Table.drop(["Ticker"], axis=1)
Balance_Sheet_Table.head()


0  Ticker_ID              Company_Name Total Cash (mrq)  \
0          1                Apple Inc.           80.09B   
1          2  American Express Company           33.28B   
2          3        The Boeing Company            7.73B   
3          4          Caterpillar Inc.            6.36B   
4          5       Cisco Systems, Inc.           40.38B   

0 Total Cash Per Share (mrq) Total Debt (mrq) Total Debt/Equity (mrq)  \
0                      17.41          112.63B                  106.39   
1                      39.85           61.37B                  276.21   
2                      13.74           15.81B                 6813.79   
3                      11.12           36.54B                  236.08   
4                       9.17           25.66B                   62.90   

0 Current Ratio (mrq) Book Value Per Share (mrq)  
0                1.32                      22.98  
1                1.88                      26.55  
2                1.07                       0.22  
3                1.43                      27.00  
4                1.80                       9.22

In [17]:
Cash_Flow_Table=df7.T.copy()
Cash_Flow_Table.columns=tables[6][0]
Cash_Flow_Table.reset_index(level=0,inplace=True)
Cash_Flow_Table=Cash_Flow_Table.rename(columns={"index":"Ticker"})
Cash_Flow_Table.insert(0, "Ticker_ID", dfcopy.index+1)
Cash_Flow_Table.insert(1,"Company_Name", Company_Name_Table["Company_Name"].copy())
Cash_Flow_Table=Cash_Flow_Table.drop(["Ticker"], axis=1)
Cash_Flow_Table.head()


0  Ticker_ID              Company_Name Operating Cash Flow (ttm)  \
0          1                Apple Inc.                    71.86B   
1          2  American Express Company                    15.41B   
2          3        The Boeing Company                    14.97B   
3          4          Caterpillar Inc.                     6.74B   
4          5       Cisco Systems, Inc.                    14.08B   

0 Levered Free Cash Flow (ttm)  
0                       45.63B  
1                          NaN  
2                        8.45B  
3                        2.56B  
4                        8.06B

In [18]:
url="https://tradingeconomics.com/united-states/stock-market"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
results2 = soup.find_all('td', class_="hidden-xs")
othertickers=[]
daily_change=[]
daily_percent_change=[]
yearly_percent_change=[]
price=[]
for x in results2:
        if x.b:
           othertickers.append(x.b.text)
othertickers=othertickers[:30]

In [19]:
results3=soup.find_all('td', id="nch")
for x in results3:
    daily_change.append(x.text.strip())
daily_change=daily_change[:30]                       
results4=soup.find_all('td', id="pch")
for x in results4:
    daily_percent_change.append(x.text.strip())
daily_percent_change=daily_percent_change[:30]
results5=soup.find_all('td')
for x in range(6,276,9):
    yearly_percent_change.append(results5[x].text)
yearly_percent_change=yearly_percent_change[:30]
results6=soup.find_all('td', id="p")
for x in results6:
    price.append(x.text.strip())
price=price[:30]
General_Table=pd.DataFrame()
General_Table["Ticker"]=othertickers
General_Table["Price"]=price
General_Table["Daily_Change"]=daily_change
General_Table["Daily_Percent_Change"]=daily_percent_change
General_Table["Yearly_Percent_Change"]=yearly_percent_change
General_Table=General_Table.sort_values(by="Ticker")
General_Table=General_Table.reset_index(drop=True)
General_Table.insert(0,"Ticker_ID", General_Table.index+1)
General_Table.head()

Ticker_ID Ticker   Price Daily_Change Daily_Percent_Change  \
0          1   AAPL  185.15       -15.57               -7.76%   
1          2    AXP  115.71        -1.55               -1.32%   
2          3     BA  337.32       -16.81               -4.75%   
3          4    CAT  125.06        -6.14               -4.68%   
4          5   CSCO   51.18        -1.74               -3.29%   

  Yearly_Percent_Change  
0                -1.73%  
1                14.11%  
2                -1.40%  
3               -19.23%  
4                11.43%

In [20]:
url="https://finance.yahoo.com/quote/HD/profile?p=HD"

In [21]:
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
newresult = soup.find_all('span', class_="Fw(600)")
newresult

[<span class="Fw(600)" data-reactid="21">Consumer Cyclical</span>,
 <span class="Fw(600)" data-reactid="25">Home Improvement Stores</span>,
 <span class="Fw(600)" data-reactid="29"><span data-reactid="30">413,000</span></span>]

In [22]:
sector=[]
industry=[]
employee_number=[]
for x in tickers:
    url=f"https://finance.yahoo.com/quote/{x}/profile?p={x}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    newresult1 = soup.find_all('span', class_="Fw(600)")
    sector.append(newresult1[0].text)
    industry.append(newresult1[1].text)
    employee_number.append(newresult1[2].text)

In [23]:
Generalinfo=pd.DataFrame()
Generalinfo["Ticker"]=tickers
Generalinfo["Industry"]=industry
Generalinfo["Sector"]=sector
Generalinfo["Numbers_of_Employee"]=employee_number
Generalinfo.insert(0,"Ticker_ID",Generalinfo.index+1)
Generalinfo.head()

Ticker_ID Ticker                       Industry              Sector  \
0          1   AAPL           Consumer Electronics          Technology   
1          2    AXP                Credit Services  Financial Services   
2          3     BA            Aerospace & Defense         Industrials   
3          4    CAT  Farm & Construction Equipment         Industrials   
4          5   CSCO        Communication Equipment          Technology   

  Numbers_of_Employee  
0             100,000  
1              59,000  
2             153,000  
3             104,000  
4              74,200

In [24]:
Industry_Table=pd.DataFrame()
Industry_Table["Industry"]=industry
Industry_Table["Sector"]=sector
Industry_Table=Industry_Table.drop_duplicates(subset='Industry', keep="last")
Industry_Table=Industry_Table.sort_values(by="Industry")
Industry_Table=Industry_Table.reset_index(drop=True)
Industry_Table.insert(0,"Industry_ID",Industry_Table.index+1)
Industry_Table
mergetable=pd.merge(Generalinfo,Industry_Table,on="Industry")
mergetable=mergetable.drop(["Sector_x","Numbers_of_Employee"],axis=1)
mergetable=mergetable.rename(columns={"Sector_y":"Sector"})
mergetable.head()

Ticker_ID Ticker              Industry  Industry_ID              Sector
0          1   AAPL  Consumer Electronics            7          Technology
1          2    AXP       Credit Services            8  Financial Services
2         26      V       Credit Services            8  Financial Services
3          3     BA   Aerospace & Defense            1         Industrials
4         25    UTX   Aerospace & Defense            1         Industrials

In [25]:
Sector_Table=pd.DataFrame()
Sector_Table["Sector"]=sector
Sector_Table=Sector_Table.drop_duplicates(subset='Sector', keep="last")
Sector_Table=Sector_Table.reset_index(drop=True)
Sector_Table.insert(0,"Sector_ID",Sector_Table.index+1)
Sector_Table.head()

Sector_ID             Sector
0          1    Basic Materials
1          2         Technology
2          3  Consumer Cyclical
3          4         Healthcare
4          5        Industrials

In [26]:
mergetable1=pd.merge(mergetable,Sector_Table,on="Sector")
mergetable1=mergetable1.sort_values(by="Ticker_ID")
mergetable1=mergetable1.reset_index(drop=True)
Stock_Type_Table=mergetable1.drop(["Industry","Sector"], axis=1)
Stock_Type_Table.head()

Ticker_ID Ticker  Industry_ID  Sector_ID
0          1   AAPL            7          2
1          2    AXP            8          6
2          3     BA            1          5
3          4    CAT           12          5
4          5   CSCO            6          2

In [27]:
Industry_Table=mergetable1[["Industry_ID","Industry","Sector_ID"]]
Industry_Table=Industry_Table.drop_duplicates(subset="Industry",keep="last")
Industry_Table=Industry_Table.sort_values(by="Industry_ID")
Industry_Table=Industry_Table.reset_index(drop=True)
Industry_Table.head(10)

Industry_ID                 Industry  Sector_ID
0            1      Aerospace & Defense          5
1            2           Banks - Global          6
2            3  Beverages - Soft Drinks          8
3            4          Capital Markets          6
4            5                Chemicals          1
5            6  Communication Equipment          2
6            7     Consumer Electronics          2
7            8          Credit Services          6
8            9          Discount Stores          8
9           10  Diversified Industrials          5

In [28]:
#Industry_Table,Stock_Type_Table,Sector_Table,General_Table

In [29]:
#Valuation_Measures_Table,Stock_Price_Hist_Table,Dividends_Splits_Table,Share_Statistics_Table,Profitability_Table,Management_Effectiveness_Table,
#Income_Statement_Table,Balance_Sheet_Table,Cash_Flow_Table

#Company_Name_Table

In [30]:
aa=[]
kk=[]
for x in tickers:
    url=f"https://money.cnn.com/quote/profile/profile.html?symb={x}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    newresult2=soup.find_all("div", class_="wsod_companyContactInfo wsod_companyPhoneURL")
    newresult = soup.find_all('div', class_="wsod_companyContactInfo wsod_companyNameStreet")
    for x in newresult:
        zz=x.find_all('div')
        for z in zz:
            aa.append(z.text)
                
    for x in newresult2:
        ff=x.find_all('div')
        for v in ff:
            kk.append(v.text)

In [31]:
aa[2]=aa[2][:-5]
aa[5]=aa[5][:-5]
aa[8]=aa[8][:-5]
aa[14]=aa[14][:-5]
aa[17]=aa[17][:-5]
aa[20]=aa[20][:-5]
aa[29]=aa[29][:-5]
aa[32]=aa[32][:-5]
aa[35]=aa[35][:-5]
aa[41]=aa[41][:-5]
aa[44]=aa[44][:-5]
aa[50]=aa[50][:-5]
aa[56]=aa[56][:-5]
aa[62]=aa[62][:-5]
aa[65]=aa[65][:-5]
aa[68]=aa[68][:-5]
aa[77]=aa[77][:-5]
aa[86]=aa[86][:-5]
aa[89]=aa[89][:-5]


In [32]:
add=[]
for x in range(1,90,3):
    add.append(aa[x]+", "+aa[x+1][:-6]+", "+aa[x+1][-5:])
add
zipcode=[]
for x in range(1,90,3):
    zipcode.append(aa[x+1][-5:])
phone=[]
for x in range(0,90,3):
    phone.append(kk[x][2:])
website=[]
for x in range(0,90,3):
    website.append(kk[x+2])

In [33]:
rawAddress_Table=pd.DataFrame()
rawAddress_Table["Ticker"]=tickers
rawAddress_Table["Address"]=add
rawAddress_Table["Zipcode"]=zipcode
rawAddress_Table.insert(0,"Ticker_ID",rawAddress_Table.index+1)
rawAddress_Table["Website"]=website
rawAddress_Table["Phone"]=phone
Company_Contact_Talbe=rawAddress_Table
Company_Contact_Talbe["Zipcode"]=pd.to_numeric(Company_Contact_Talbe["Zipcode"])
Company_Contact_Talbe.head()

Ticker_ID Ticker                                            Address  \
0          1   AAPL    One Apple Park Way, Cupertino California, 95014   
1          2    AXP         200 Vesey Street, New York New York, 10285   
2          3     BA  100 North Riverside Plaza, Chicago Illinois, 6...   
3          4    CAT      510 Lake Cook Road, Deerfield Illinois, 60015   
4          5   CSCO  170 West Tasman Drive, San Jose California, 95134   

   Zipcode                  Website           Phone  
0    95014            www.apple.com  (408) 996-1010  
1    10285  www.americanexpress.com  (212) 640-2000  
2    60606           www.boeing.com  (312) 544-2000  
3    60015      www.caterpillar.com  (224) 551-4000  
4    95134            www.cisco.com  (408) 526-4000

In [34]:
uszip=pd.read_csv("uszips.csv",encoding="utf-8")
newcol=['city', 'state_id', 'state_name','zip', 'lat', 'lng']
uszip=uszip[newcol]
Zip_Table=uszip
Zip_Table.tail()

city state_id   state_name    zip      lat       lng
0             Adjuntas       PR  Puerto Rico    601  18.1800  -66.7522
1               Aguada       PR  Puerto Rico    602  18.3607  -67.1752
2            Aguadilla       PR  Puerto Rico    603  18.4544  -67.1220
3              Maricao       PR  Puerto Rico    606  18.1672  -66.9383
4               Anasco       PR  Puerto Rico    610  18.2903  -67.1224
5              Arecibo       PR  Puerto Rico    612  18.4070  -66.7080
6             Bajadero       PR  Puerto Rico    616  18.4175  -66.6681
7          Barceloneta       PR  Puerto Rico    617  18.4412  -66.5592
8             Boqueron       PR  Puerto Rico    622  17.9917  -67.1525
9            Cabo Rojo       PR  Puerto Rico    623  18.0835  -67.1542
10            Penuelas       PR  Puerto Rico    624  18.0646  -66.7182
11               Camuy       PR  Puerto Rico    627  18.4190  -66.8603
12            Castaner       PR  Puerto Rico    631  18.1856  -66.8333
13       Sabana Grande       PR  Puerto Rico    637  18.0798  -66.9427
14              Ciales       PR  Puerto Rico    638  18.2890  -66.5151
15              Utuado       PR  Puerto Rico    641  18.2641  -66.7138
16              Dorado       PR  Puerto Rico    646  18.4340  -66.2833
17            Ensenada       PR  Puerto Rico    647  17.9661  -66.9438
18             Florida       PR  Puerto Rico    650  18.3432  -66.5860
19         Garrochales       PR  Puerto Rico    652  18.4554  -66.6063
20             Guanica       PR  Puerto Rico    653  17.9857  -66.8903
21          Guayanilla       PR  Puerto Rico    656  18.0528  -66.7918
22             Hatillo       PR  Puerto Rico    659  18.4107  -66.7965
23         Hormigueros       PR  Puerto Rico    660  18.1342  -67.1140
24             Isabela       PR  Puerto Rico    662  18.4621  -67.0144
25              Jayuya       PR  Puerto Rico    664  18.2115  -66.5889
26               Lajas       PR  Puerto Rico    667  18.0123  -67.0409
27               Lares       PR  Puerto Rico    669  18.2710  -66.8698
28          Las Marias       PR  Puerto Rico    670  18.2378  -66.9887
29              Manati       PR  Puerto Rico    674  18.4212  -66.4900
...                ...      ...          ...    ...      ...       ...
33069           Ambler       AK       Alaska  99786  67.1732 -155.9542
33070      Chalkyitsik       AK       Alaska  99788  66.6463 -143.7815
33071          Nuiqsut       AK       Alaska  99789  70.0298 -151.8612
33072        Fairbanks       AK       Alaska  99790  65.4213 -148.2880
33073          Atqasuk       AK       Alaska  99791  70.5610 -156.6387
33074           Juneau       AK       Alaska  99801  58.3986 -134.0578
33075           Angoon       AK       Alaska  99820  57.4401 -134.1968
33076          Douglas       AK       Alaska  99824  58.2724 -134.4008
33077       Elfin Cove       AK       Alaska  99825  58.1360 -135.8726
33078         Gustavus       AK       Alaska  99826  58.7512 -136.5673
33079           Haines       AK       Alaska  99827  59.1222 -135.7064
33080           Hoonah       AK       Alaska  99829  58.1074 -135.4258
33081             Kake       AK       Alaska  99830  56.7790 -134.1603
33082          Pelican       AK       Alaska  99832  58.0162 -136.1806
33083       Petersburg       AK       Alaska  99833  56.8844 -133.0817
33084            Sitka       AK       Alaska  99835  56.9606 -135.0545
33085   Port Alexander       AK       Alaska  99836  56.2353 -134.6534
33086          Skagway       AK       Alaska  99840  59.5617 -135.3374
33087  Tenakee Springs       AK       Alaska  99841  57.7943 -135.1849
33088        Ketchikan       AK       Alaska  99901  55.5693 -131.0130
33089     Meyers Chuck       AK       Alaska  99903  55.8214 -132.0593
33090     Coffman Cove       AK       Alaska  99918  55.9467 -132.7437
33091       Thorne Bay       AK       Alaska  99919  55.6508 -132.5034
33092            Craig       AK       Alaska  99921  55.4548 -132.7721
33093         Hydaburg       AK       Alas

In [35]:
db = MySQLdb.connect(host="localhost",    # your host, usually localhost
                     user=f"{user_name}",         # your username
                     passwd=f"{password}",  # your password
                     use_unicode=True,
                     charset="utf8"
                     
                        )        # name of the data base
cursor = db.cursor()


PROJECT_DB="ETL_STOCK_DB"
# Execute SQL select statement
cursor.execute(f"DROP DATABASE IF EXISTS {PROJECT_DB}")

cursor.execute(f"CREATE DATABASE IF NOT EXISTS {PROJECT_DB}")
cursor.close()
db = MySQLdb.connect(host="localhost",    # your host, usually localhost
                     user=f"{user_name}",         # your username
                     passwd=f"{password}",  # your password
                     use_unicode=True,
                     charset="utf8",
                     database=f"{PROJECT_DB}"
                        )        # name of the data base
cursor = db.cursor()



In [36]:
rds_connection_string = f"{user_name}:{password}@127.0.0.1/{PROJECT_DB}"
engine = create_engine(f'mysql://{rds_connection_string}?charset=utf8')

In [37]:
#Valuation_Measures_Table,Stock_Price_Hist_Table,Dividends_Splits_Table,Share_Statistics_Table,Profitability_Table,Management_Effectiveness_Table,
#Income_Statement_Table,Balance_Sheet_Table,Cash_Flow_Table

#Company_Name_Table

#Industry_Table,Stock_Type_Table,Sector_Table,General_Table

In [38]:
General_Table.to_sql(name='stock_summary', con=engine, if_exists='replace', index=False)
cursor.execute('Alter Table `stock_summary` ADD PRIMARY KEY (`TICKER_ID`)')

Valuation_Measures_Table.to_sql(name='valuation_measures', con=engine, if_exists='replace', index=False)
cursor.execute('Alter Table `valuation_measures` ADD PRIMARY KEY (`TICKER_ID`)')

Stock_Price_Hist_Table.to_sql(name='price_history', con=engine, if_exists='replace', index=False)
cursor.execute('Alter Table `price_history` ADD PRIMARY KEY (`TICKER_ID`)')

Dividends_Splits_Table.to_sql(name='dividends_splits', con=engine, if_exists='replace', index=False)
cursor.execute('Alter Table `dividends_splits` ADD PRIMARY KEY (`TICKER_ID`)')

Share_Statistics_Table.to_sql(name='share_stats', con=engine, if_exists='replace', index=False)
cursor.execute('Alter Table `share_stats` ADD PRIMARY KEY (`TICKER_ID`)')

Profitability_Table.to_sql(name='profitability', con=engine, if_exists='replace', index=False)
cursor.execute('Alter Table `profitability` ADD PRIMARY KEY (`TICKER_ID`)')

Management_Effectiveness_Table.to_sql(name='manage_effectiveness', con=engine, if_exists='replace', index=False)
cursor.execute('Alter Table `manage_effectiveness` ADD PRIMARY KEY (`TICKER_ID`)')

Income_Statement_Table.to_sql(name='income_statement', con=engine, if_exists='replace', index=False)
cursor.execute('Alter Table `income_statement` ADD PRIMARY KEY (`TICKER_ID`)')

Balance_Sheet_Table.to_sql(name='balance_sheet', con=engine, if_exists='replace', index=False)
cursor.execute('Alter Table `balance_sheet` ADD PRIMARY KEY (`TICKER_ID`)')

Cash_Flow_Table.to_sql(name='cash_flow', con=engine, if_exists='replace', index=False)
cursor.execute('Alter Table `cash_flow` ADD PRIMARY KEY (`TICKER_ID`)')


0

In [39]:
Company_Name_Table.to_sql(name='company_name', con=engine, if_exists='replace', index=False)
cursor.execute('Alter Table `company_name` ADD PRIMARY KEY (`TICKER_ID`)')

Company_Contact_Talbe.to_sql(name='contact_info', con=engine, if_exists='replace', index=False)
cursor.execute('Alter Table `contact_info` ADD PRIMARY KEY (`TICKER_ID`)')

Zip_Table.to_sql(name='zipcode', con=engine, if_exists='replace', index=False)
cursor.execute('Alter Table `zipcode` ADD PRIMARY KEY (`zip`)')

0

In [40]:
Industry_Table.to_sql(name='industry', con=engine, if_exists='replace', index=False)
cursor.execute('Alter Table `industry` ADD PRIMARY KEY (`industry_id`)')

Stock_Type_Table.to_sql(name='profile_info', con=engine, if_exists='replace', index=False)
cursor.execute('Alter Table `profile_info` ADD PRIMARY KEY (`ticker_id`)')

Sector_Table.to_sql(name='sector', con=engine, if_exists='replace', index=False)
cursor.execute('Alter Table `sector` ADD PRIMARY KEY (`sector_id`)')

Zip_Table.to_sql(name='zipcode', con=engine, if_exists='replace', index=False)
cursor.execute('Alter Table `zipcode` ADD PRIMARY KEY (`zip`)')

0

In [41]:
cursor.execute('ALTER TABLE `stock_summary` ADD FOREIGN KEY (`ticker_id`) REFERENCES `company_name`(`ticker_id`) ON DELETE CASCADE')
cursor.execute('ALTER TABLE `contact_info` ADD FOREIGN KEY (`ticker_id`) REFERENCES `company_name`(`ticker_id`) ON DELETE CASCADE')
cursor.execute('ALTER TABLE `profile_info` ADD FOREIGN KEY (`ticker_id`) REFERENCES `company_name`(`ticker_id`) ON DELETE CASCADE')
cursor.execute('ALTER TABLE `profile_info` ADD FOREIGN KEY (`industry_id`) REFERENCES `industry`(`industry_id`) ON DELETE CASCADE')
cursor.execute('ALTER TABLE `industry` ADD FOREIGN KEY (`sector_id`) REFERENCES `sector`(`sector_id`) ON DELETE CASCADE')


25

In [42]:
cursor.execute('ALTER TABLE `valuation_measures` ADD FOREIGN KEY (`ticker_id`) REFERENCES `stock_summary`(`ticker_id`) ON DELETE CASCADE')
cursor.execute('ALTER TABLE `price_history` ADD FOREIGN KEY (`ticker_id`) REFERENCES `stock_summary`(`ticker_id`) ON DELETE CASCADE')
cursor.execute('ALTER TABLE `dividends_splits` ADD FOREIGN KEY (`ticker_id`) REFERENCES `stock_summary`(`ticker_id`) ON DELETE CASCADE')
cursor.execute('ALTER TABLE `share_stats` ADD FOREIGN KEY (`ticker_id`) REFERENCES `stock_summary`(`ticker_id`) ON DELETE CASCADE')
cursor.execute('ALTER TABLE `profitability` ADD FOREIGN KEY (`ticker_id`) REFERENCES `stock_summary`(`ticker_id`) ON DELETE CASCADE')
cursor.execute('ALTER TABLE `manage_effectiveness` ADD FOREIGN KEY (`ticker_id`) REFERENCES `stock_summary`(`ticker_id`) ON DELETE CASCADE')
cursor.execute('ALTER TABLE `income_statement` ADD FOREIGN KEY (`ticker_id`) REFERENCES `stock_summary`(`ticker_id`) ON DELETE CASCADE')
cursor.execute('ALTER TABLE `balance_sheet` ADD FOREIGN KEY (`ticker_id`) REFERENCES `stock_summary`(`ticker_id`) ON DELETE CASCADE')
cursor.execute('ALTER TABLE `cash_flow` ADD FOREIGN KEY (`ticker_id`) REFERENCES `stock_summary`(`ticker_id`) ON DELETE CASCADE')


30